# Homework part A: Hard Spheres

Go to https://github.com/joaander/hoomd-che629 for instructions on how to modify and execute this homework.

## Boilerplate code

In [ ]:
# This cell installs HOOMD in Google Colab. Delete it if you run locally
!pip install -q condacolab
import condacolab
condacolab.install_from_url('https://github.com/joaander/hoomd-che629/releases/download/2022.0.0/hoomd-che629-2022.0-Linux-x86_64.sh')

In [ ]:
import hoomd
import math
import itertools
import numpy
import copy
import gsd.hoomd
import freud
import matplotlib
import IPython
import os
%matplotlib inline
matplotlib.style.use('ggplot')

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=250, h=250)

def render(snapshot, params):
    L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=snapshot.particles.N,
                                       radius=1/2)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(0, 0, L+1),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L+1)
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=100)

## Workflow stages

The tutorial explained all of the stages of a hard sphere self-assembly simulation in individual notebooks.
Let's collect all the code from those notebooks together here in functions.
You can call these to explore simulation results as you change parameters or perform additional analysis.
First, you need a dictionary to describe the parameters of the simulation:
* `phi`: Volume fraction in the simulation box.
* `seed`: Random number seed for MC simulation.
* `N_particles`: Number of particles to place in the simulation box.
* `directory`: Directory to store output files.

You can use separate directories to store the output of many simulations for combined analysis.

In [ ]:
example_params = dict(phi=0.57, seed=3, N_particles=500, directory='phi57_seed3')

You will pass this dictionary into each function that needs it.

The first function makes the HPMC **Sphere** integrator based on the given parameters:

In [ ]:
def make_hard_sphere_integrator(params):
    mc = hoomd.hpmc.integrate.Sphere()
    mc.shape['sphere'] = dict(diameter=1)
    return mc

The next function places N particles on a cubic lattice and writes out `lattice.gsd`:

In [ ]:
def initialize(params):
    os.makedirs(params['directory'], exist_ok=True)
    
    spacing = 1.1
    K = math.ceil(params['N_particles']**(1/3))
    L = K * spacing
    x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
    position = list(itertools.product(x, repeat=3))
    position = position[0:params['N_particles']]
    
    snapshot = gsd.hoomd.Snapshot()
    snapshot.particles.N = params['N_particles']
    snapshot.particles.position = position
    snapshot.particles.typeid = [0]*params['N_particles']
    snapshot.particles.types = ['sphere']
    snapshot.configuration.box = [L, L, L, 0, 0, 0]
    
    with gsd.hoomd.open(name=params['directory'] +  '/lattice.gsd', mode='wb') as f:
        f.append(snapshot)

The next function reads in `lattice.gsd` and runs HPMC simulations to randomize the positions of the particles and writes `random.gsd`:

In [ ]:
def randomize(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu, seed=params['seed'])
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename=params['directory'] + '/lattice.gsd')
    sim.run(10e3)
    hoomd.write.GSD.write(state=sim.state, filename=params['directory'] + '/random.gsd')

The next function reads in `random.gsd` and compresses the system to the target volume fraction and writes `compressed.gsd`:

In [ ]:
def compress(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu, seed=params['seed'])
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename=params['directory'] + '/random.gsd')

    V_particle = 4/3 * math.pi * (1/2)**3
    
    initial_box = sim.state.box
    final_box = hoomd.Box.from_box(initial_box)
    final_box.volume = sim.state.N_particles * V_particle / params['phi']
    compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10), target_box = final_box)
    sim.operations.updaters.append(compress)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['d'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.Periodic(10),
                                                 max_translation_move=0.2)
    sim.operations.tuners.append(tune)   
    
    while not compress.complete and sim.timestep < 1e6:
        sim.run(1000)
    
    if not compress.complete:
        raise RuntimeError("Compression failed to complete")
    
    hoomd.write.GSD.write(state=sim.state, filename=params['directory'] + '/compressed.gsd')

The next function reads in `compressed.gsd` and equilibrates the system over many steps, writing out `trajectory.gsd`:

In [ ]:
def equilibrate(params):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu, seed=params['seed'])
    sim.operations.integrator = make_hard_sphere_integrator(params)
    sim.create_state_from_gsd(filename=params['directory'] + '/compressed.gsd')

    gsd = hoomd.write.GSD(filename=params['directory'] + '/trajectory.gsd',
                          trigger=hoomd.trigger.Periodic(1000),
                          mode='wb')
    sim.operations.writers.append(gsd)
    
    tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                                 target=0.2,
                                                 trigger=hoomd.trigger.And(
                                                     [hoomd.trigger.Periodic(100),
                                                      hoomd.trigger.Before(sim.timestep + 5000)]))
    sim.operations.tuners.append(tune)   

    logger = hoomd.logging.Logger(categories=['scalar', 'string'])
    logger.add(sim, quantities=['timestep', 'final_timestep', 'tps'])
    table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=10000), logger=logger)
    sim.operations.writers.append(table)

    sim.run(100e3)

The next function analyzes the given trajectory and plots the number of particles in a solid-like environment over time:

In [ ]:
def analyze_nsolid(params):
    with gsd.hoomd.open(params['directory'] + '/trajectory.gsd') as traj:
        solid = freud.order.SolidLiquid(l=6, q_threshold=0.7, solid_threshold=6)
        is_solid = []
        for frame in traj:
            solid.compute(system=(frame.configuration.box, frame.particles.position),
                          neighbors=dict(mode='nearest', num_neighbors=8))
            is_solid.append(solid.num_connections > solid.solid_threshold)  
            
    fig = matplotlib.figure.Figure(figsize=(10, 6.18))
    ax = fig.add_subplot()
    num_solid = [numpy.sum(a) for a in is_solid]
    ax.plot(num_solid)
    ax.set_xlabel('frame')
    ax.set_ylabel('number of particles in a solid environment')
    ax.set_ylim([0, params['N_particles']])
    IPython.display.display(fig)

The next function visualizes the last frame of the trajectory:

In [ ]:
def visualize(params):
    with gsd.hoomd.open(params['directory'] + '/trajectory.gsd') as traj:
        IPython.display.display(render(traj[-1], params))

The final function combines the `initialize`, `randomize`, `compress`, and `equilibrate` steps into one function.

In [ ]:
def run_simulation(params):
    initialize(params)
    randomize(params)
    compress(params)
    equilibrate(params)

## Run the simulation

Call the `run_simulation` function defined above to run the whole simulation with the given parameters:

<div class="alert alert-warning">
This cell will take a few minutes to complete.
</div>

In [ ]:
sim_params = dict(phi=0.57, seed=1, N_particles=500, directory='phi57_seed3')
run_simulation(sim_params)

Analyze the results of the simulation:

In [ ]:
visualize(sim_params)

In [ ]:
analyze_nsolid(sim_params)

# Exercises

1. Run the simulation at phi=0.57 for 5 different random seeds. Plot the solid analysis and visualize these simulation results.


In [ ]:
# add as many code blocks as you need

- Describe the effect of changing the seed.
- Do all seeds lead to the same final structure?
- Do any of your seeds result in a simulation where all ~500 particles in the box end in a solid-like environment?

**Answer:** *Enter your answer here.*

2. Run the simulation for 5 volume fractions between phi=0.50 and phi=0.58. Plot the solid analysis and visualize these simulation results.

In [ ]:
# add as many code blocks as you need

**Questions**
- How does the system behave at lower volume fractions (phi)?
- at what volume fraction does the system start to crystallize?

**Answer:** *Enter your answer here.*

3.  Write a new analysis method to compute the average [Steinhardt $q_6$ order parameter using freud](https://freud.readthedocs.io/en/stable/modules/order.html#freud.order.Steinhardt).
 Plot the average $q_6$ in the last frame of the trajectory vs `phi` (*used the saved simulation results from exercise 2*).

In [ ]:
# add as many code blocks as you need

**Questions**

- At what density does the system start to order?

**Answer:** *Enter your answer here.*

4. This notebook initializes the system in a random configuration and allows it to crystallize. 
You can also use simulations to explore melting behavior and the (meta-)stability of particular structures.
    Remove the **compression** step from the simulation and modify the **initialization** step to place particles on a perfect fcc (or bcc) structure at the target volume fraction.

In [ ]:
# add as many code blocks as you need

**Questions**

- At what volume fraction does the system melt into a fluid? 
- Is the the same volume fraction that the fluid crystallizes into the solid?

**Answer:** *Enter your answer here.*


## BONUS:

Open some trajectory files in [OVITO](https://www.ovito.org/). Use the [polyhedral template matching](https://www.ovito.org/docs/current/particles.modifiers.polyhedral_template_matching.php) modifier to color particles by local environment and visualize the types of defects in the system.

[Previous section](00-index.ipynb) / [Next section](02-Hard-Sphereocylinders.ipynb)